# Merging features for the first model

Merging: BDI weekly data with CGDP weekly, CIP weekly, and CSTEEL weekly

In [40]:
import numpy as np
import pandas as pd

In [41]:
csteel_df = pd.read_csv("../raw_data/data/CSTEEL/cleaned_weekly_CSTEEL.csv")
cgdp_df = pd.read_csv("../raw_data/data/CGDP/cleaned_weekly_CGDP.csv")
cip_df = pd.read_csv("../raw_data/data/CIP/cleaned_weekly_CIP.csv")

## Initial setup

In [62]:
csteel_df.tail(2)

,time,close,csteel
1390,2021-11-22,2767.0,8.113000e+07
1391,2021-11-29,3171.0,8.507333e+07


In [65]:
cgdp_df.tail(2)

,time,close,cgdp,diff,cgdp_growth
1390,2021-11-22,2767.0,1.765894e+13,5.842392e+10,0.003308
1391,2021-11-29,3171.0,1.771737e+13,5.842392e+10,0.003298


In [66]:
cip_df = cip_df[cip_df.index < "2021-11-30"]

In [67]:
cip_df.tail(2)

,CIP
time,
2021-11-28,4.193864
2021-11-29,4.227298


## Creating a dataframe of CIP values that only has the CIP values when there is a weekly BDI value:

In [68]:
bdi_df = pd.read_csv("../raw_data/data/BDI/cleaned_weekly_BDI.csv")

In [69]:
bdi_df.tail(2)

,time,close
1439,2022-11-07,1355.0
1440,2022-11-14,1189.0


In [71]:
bdi_df = bdi_df[bdi_df["time"] < "2021-11-30"]

In [73]:
bdi_df.tail(2)

,time,close
1390,2021-11-22,2767.0
1391,2021-11-29,3171.0


In [74]:
bdi_df["time"] = pd.to_datetime(bdi_df["time"])

In [75]:
bdi_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1392 entries, 0 to 1391
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    1392 non-null   datetime64[ns]
 1   close   1392 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 32.6 KB


In [76]:
bdi_df.set_index("time", inplace=True)

In [78]:
bdi_df.tail(2)

,close
time,
2021-11-22,2767.0
2021-11-29,3171.0


In [79]:
cip_df.tail(2)

,CIP
time,
2021-11-28,4.193864
2021-11-29,4.227298


In [80]:
cip_df["time"] = pd.to_datetime(cip_df["time"])

KeyError: 'time'

In [54]:
cip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9832 entries, 0 to 9831
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    9832 non-null   datetime64[ns]
 1   CIP     9832 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 153.8 KB


In [55]:
cip_df.set_index("time", inplace=True)

In [56]:
cip_df.head(2)

,CIP
time,
1995-01-01,11.400000
1995-01-02,11.601218


In [83]:
cip_bdi_combined_df = pd.merge(bdi_df, cip_df, on="time", how="inner")

In [85]:
cip_bdi_combined_df.tail(2)

,close,CIP
time,,
2021-11-22,2767.0,4.030326
2021-11-29,3171.0,4.227298


# Combining all the cleaned features together into one dataframe

### combining cgdp_df and csteel_df

In [89]:
cgdp_df.head(2)

,time,close,cgdp,diff,cgdp_growth
0,1995-01-16,2059.0,5.857736e+11,NaN,NaN
1,1995-01-23,1983.0,5.890383e+11,3.264685e+09,0.005542


In [90]:
csteel_df.head(2)

,time,close,csteel
0,1995-01-16,2059.0,7.380742e+06
1,1995-01-23,1983.0,7.349355e+06


In [92]:
csteel_cgdp_df = pd.merge(cgdp_df, csteel_df, how="inner", on="time")

In [99]:
csteel_cgdp_df.drop(columns="close_y", inplace=True)

In [102]:
csteel_cgdp_df.rename(columns={"close_x":"bdi"}, inplace=True)

In [108]:
csteel_cgdp_df["time"] = pd.to_datetime(csteel_cgdp_df["time"])

In [110]:
csteel_cgdp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1392 entries, 0 to 1391
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   time         1392 non-null   datetime64[ns]
 1   bdi          1392 non-null   float64       
 2   cgdp         1392 non-null   float64       
 3   diff         1391 non-null   float64       
 4   cgdp_growth  1391 non-null   float64       
 5   csteel       1392 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 108.4 KB


### Combining csteel_cgdp_df with cip

In [136]:
final_df = pd.merge(csteel_cgdp_df, cip_bdi_combined_df, on="time", how="inner")

In [137]:
final_df.drop(columns="close")

,time,bdi,cgdp,diff,cgdp_growth,csteel,CIP
0,1995-01-16,2059.0,5.857736e+11,NaN,NaN,7.380742e+06,13.900686
1,1995-01-23,1983.0,5.890383e+11,3.264685e+09,0.005542,7.349355e+06,14.721669
2,1995-01-30,1959.0,5.923030e+11,3.264685e+09,0.005512,7.317968e+06,15.351956
3,1995-02-06,1980.0,5.955677e+11,3.264685e+09,0.005482,7.418464e+06,15.812899
4,1995-02-13,1992.0,5.988324e+11,3.264685e+09,0.005452,7.571714e+06,16.125851
...,...,...,...,...,...,...,...
1387,2021-11-01,2715.0,1.748367e+13,5.842392e+10,0.003342,6.930000e+07,3.800000
1388,2021-11-08,2807.0,1.754210e+13,5.842392e+10,0.003330,7.324333e+07,3.837203
1389,2021-11-15,2552.0,1.760052e+13,5.842392e+10,0.003319,7.718667e+07,3.907136
1390,2021-11-22,2767.0,1.765894e+13,5.842392e+10,0.003308,8.113000e+07,4.030326


In [138]:
final_df.set_index("time", inplace=True)

In [150]:
final_df.drop(columns="close", inplace=True)

In [151]:
final_df

,bdi,cgdp,diff,cgdp_growth,csteel,CIP
time,,,,,,
1995-01-16,2059.0,5.857736e+11,NaN,NaN,7.380742e+06,13.900686
1995-01-23,1983.0,5.890383e+11,3.264685e+09,0.005542,7.349355e+06,14.721669
1995-01-30,1959.0,5.923030e+11,3.264685e+09,0.005512,7.317968e+06,15.351956
1995-02-06,1980.0,5.955677e+11,3.264685e+09,0.005482,7.418464e+06,15.812899
1995-02-13,1992.0,5.988324e+11,3.264685e+09,0.005452,7.571714e+06,16.125851
...,...,...,...,...,...,...
2021-11-01,2715.0,1.748367e+13,5.842392e+10,0.003342,6.930000e+07,3.800000
2021-11-08,2807.0,1.754210e+13,5.842392e+10,0.003330,7.324333e+07,3.837203
2021-11-15,2552.0,1.760052e+13,5.842392e+10,0.003319,7.718667e+07,3.907136


### Applying a standard scaler to the final_df

In [152]:
from sklearn.preprocessing import StandardScaler

In [153]:
scaler = StandardScaler()

In [154]:
X = scaler.fit_transform(final_df)

In [155]:
X = pd.DataFrame(X)

In [156]:
X.set_index(final_df.index, inplace=True)

In [157]:
X

,0,1,2,3,4,5
time,,,,,,
1995-01-16,-0.030272,-1.060914,NaN,NaN,-1.296075,0.542656
1995-01-23,-0.071166,-1.060265,-0.710949,2.303236,-1.297217,0.700929
1995-01-30,-0.084080,-1.059616,-0.710949,2.280504,-1.298359,0.822439
1995-02-06,-0.072780,-1.058967,-0.710949,2.258022,-1.294703,0.911302
1995-02-13,-0.066323,-1.058318,-0.710949,2.235784,-1.289127,0.971635
...,...,...,...,...,...,...
2021-11-01,0.322703,2.298850,3.621600,0.665619,0.956718,-1.404607
2021-11-08,0.372205,2.310467,3.621600,0.657337,1.100188,-1.397435
2021-11-15,0.234997,2.322083,3.621600,0.649111,1.243657,-1.383953


In [158]:
final_df.to_csv("../raw_data/data/merged_data_prescaling.csv")

In [159]:
X.to_csv("../raw_data/data/merged_data_postscaling.csv")